# Project Milestone2 - Group Concatsanddogs

# The use of women's rights and gender equality rhetoric in the US

## Header

This notebook is organized followings the different steps used in our pipeline. We first create a list of keywords using [web scraping](#Webscraping) and a personal list of keywords. 
With this list of keywords we [select](#Dataset-selection-from-Quotebank-database) a subset of the Quotebank database. This subset will be our starting dataset for our project. 

## General librairies

In [5]:
import pandas as pd
import numpy as np

## Webscraping

AMINA TO ADD

### Libraries

In [12]:
import requests #http library
import nltk #natural language processing library
nltk.download('stopwords') #common english words to ignore 
from bs4 import BeautifulSoup #extraction from HTML and XML files
from collections import Counter #dictionary subclass for counting hashable objects

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aminamatt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Functions

### Initialization

In [ ]:
#list of primary links containing articles of interest
URL_TOPIC_LIST = ['https://www.usnews.com/topics/subjects/feminism',
             'https://www.usnews.com/topics/subjects/gender',
              'https://www.usnews.com/topics/subjects/gender_bias',
             'https://www.usnews.com/topics/subjects/sexism']

### Retrieving of articles of interest

In [ ]:
all_articles = ''

for url_topic in URL_TOPIC_LIST:
    #Retrieve all urls for latest articles in the chosen topic pages
    usnews_topic_urls = get_urls_usnews(url_topic)
    
    #Retrieve all the articles contents for the latest articles
    all_articles_topic =  get_all_articles(usnews_topic_urls)
    
    #append articles to create one text
    all_articles = all_articles +' '+all_articles_topic

In [ ]:
#Export all the articles of interest in a single text file
text_file = open("Articles-Contents.txt", "w")
text_file.write(all_articles)
text_file.close()

### Frequency computation for bigrams 

In [ ]:
#Couting bigram frequencies for all articles of interest
usNewsFEMbigramFreq = ngram_frequency(all_articles)

MAX = 150

#Visualize the most common bigrams
for word, frequency in usNewsFEMbigramFreq.most_common(MAX):
        print('%s;%d' % (word, frequency))

The most common bigrams list also contain a lot of Named Entities (NE) like cities, persons etc... 
Here we use the naive approach to ignore this name by using the word capitalization to select them. Note that there are more advanced way to recognize NE but we believe that it will be overkilled for our usage.

### Final List

In [14]:
bigram_final_list = []
MAX = 100
for word, frequency in usNewsFEMbigramFreq.most_common(MAX):
    if (word[0][0].isupper()==False and word[1][0].isupper()==False): #ignore the Named Entities
        bigram_final_list.append(word[0]+' '+word[1])

bigram_final_list

NameError: name 'usNewsFEMbigramFreq' is not defined

In [ ]:
### ToDo: Add comments about the one to avoid
### ToDo: drop the useless one

In [ ]:
#Add my own vocabulary
my_bigrams = ['women\'s right','Equal opportunities','Equal rights','Equal status',
           'equal pay','gender gap','Gender discrimination','Gender equality','Sexual harrasment','Women empowerment',
            'women victim','women immigration','Women emancipation','women\'s participation','Western women','non-western woman',
              'Muslim women', 'Equal wages','Gender equality',
             'gender equity','Men and women', 'women and men', 'women oppression', 'niqab ban'
           'struggle of girls','struggle of women', 'war against women','oppression of girls','oppression of women',
           'women oppression','women\'s opression','liberate women','religious oppresion',
           'abuse of women','Male oppression','Female oppression','Exploitation of women',
           'Indigenous women','Patriarchal culture']

all_bigrams = my_bigrams + bigram_final_list
all_bigrams

In [ ]:
#Add the export

## Dataset selection from Quotebank database

### Librairies

In [2]:
import os 

### Functions

In [3]:
#Processing on chunk
#Input
#Output
def process_chunk(chunk, vocabulary):
    print(f'Processing chunk with {len(chunk)} rows')
    #print(chunk.columns)
    occurences = np.zeros(len(vocabulary))
    for index, word in enumerate(vocabulary):
        occurences[index] = np.sum(chunk['quotation'].str.contains(word)) 
    return occurences

#Select quotes containing keywords
def select_quotes_chunk(chunk, keywords):
    print(f'Processing chunk with {len(chunk)} rows')
    return chunk[chunk['quotation'].str.contains('|'.join(keywords))]

#Use the selection function on each chunk of the full dataset 
def select_quotes_one_year(path_to_file, vocabulary, chunksize = 10 ** 4):
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for index, chunk in enumerate(df_reader):
            if not index==0:
                selected_df = pd.concat([selected_df, select_quotes_chunk(chunk, vocabulary)])
            else: 
                selected_df = select_quotes_chunk(chunk, vocabulary)
    return selected_df

#Use the selection function on each chunk of the full dataset 
#Dumps the selected quotes into a new json file
def select_and_dump(path_to_file, vocabulary, chunksize = 10 ** 4, year = 'replace_me'):
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for index, chunk in enumerate(df_reader):
            #Dump selected quotes
            selected_df = select_quotes_chunk(chunk, vocabulary)
            pickle_file_name = year + '_chunk_' + str(index) + '.pkl'
            selected_df.to_pickle('files/'+pickle_file_name)
            #if not index==0:
                #selected_df = pd.concat([selected_df, select_quotes_chunk(chunk, vocabulary)])
            #else: 
               # selected_df = select_quotes_chunk(chunk, vocabulary)
    return selected_df


import random, string

def randomword(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

### Constants 

In [ ]:
#Add the import of keywords

In [12]:
DATA_FOLDER = 'data/'
QUOTEBANK_2020 = DATA_FOLDER+ "quotes-2020.json.bz2"
QUOTEBANK_2019 = DATA_FOLDER+ "quotes-2019.json.bz2"
QUOTEBANK_2017 = DATA_FOLDER+ "quotes-2017.json.bz2"
QUOTEBANK_2015 = DATA_FOLDER+ "quotes-2015.json.bz2"
QUOTEBANK_2018 = DATA_FOLDER+ "quotes-2018.json.bz2"
QUOTEBANK_2016 = DATA_FOLDER+ "quotes-2016.json.bz2"

KEYWORDS_LIST = ('women\'s right','Equal opportunities','Equal rights','Equal status','equal pay',
              'gender gap','Gender discrimination','Gender equality','Sexual harrassment',
              'Women empowerment','women victim','women immigration','Women emancipation',
              'women\'s participation','Western women','non-western woman','Muslim women',
              'Equal wages','Gender equality','gender equity','Men and women','women and men',
              'women oppression','niqab ban','struggle of girls','struggle of women','war against women',
              'oppression of girls','oppression of women','women oppression','women\'s opression','liberate women',
              'religious oppresion','abuse of women','Male oppression','Female oppression','Exploitation of women',
              'Indigenous women','Patriarchal culture','gender equality','child care','men pay','percentage men',
              'pay percentage','sexual harassment','women girls','girls women',
              'rates women','women according','female mayors','share women','women movement',
              'see women','gender stereotypes','gender gap',
              'women representation','sex discrimination','states women','lose weight',
              'women rights','woman time',
              'based gender',
              'proportional electoral','female candidates','gender-based violence','entirely female','cities female')

### Select and pickle of quotes of interest

Note: This code has to be run once to create the pickle files containing the quotes of interest. For futher use, the dataframe is direcly loaded from the pickle files.

In [2]:
# %time QOI_2015_DF = select_quotes_one_year(QUOTEBANK_2015,KEYWORDS_LIST,10 ** 4)
# %time QOI_2016_DF = select_quotes_one_year(QUOTEBANK_2016,KEYWORDS_LIST,10 ** 4)
# %time QOI_2017_DF = select_quotes_one_year(QUOTEBANK_2017,KEYWORDS_LIST,10 ** 4)
# %time QOI_2018_DF = select_quotes_one_year(QUOTEBANK_2018,KEYWORDS_LIST,10 ** 4)
# %time QOI_2019_DF = select_quotes_one_year(QUOTEBANK_2019,KEYWORDS_LIST,10 ** 4)
# %time QOI_2020_DF = select_quotes_one_year(QUOTEBANK_2020,KEYWORDS_LIST,10 ** 4)

dataframesNames = ('QOI_2015_DF','QOI_2016_DF','QOI_2017_DF','QOI_2018_DF','QOI_2019_DF','QOI_2020_DF')
#dataframes = (QOI_2015_DF,QOI_2016_DF,QOI_2017_DF,QOI_2018_DF,QOI_2019_DF,QOI_2020_DF)

#for i in range(len(dataframesNames)):
#    dataframes[i].to_pickle('generated_data/'+dataframesNames[i]+'.pkl')

### Load dataframe of selected quotes

In [3]:
PATH = 'generated_data/'

In [6]:
# Concatenate into one dataframes the dataframes from each pickle file. 
df = pd.concat([pd.read_pickle(PATH+ fp +'.pkl') for fp in dataframesNames], ignore_index=True)

In [7]:
df.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2015-03-09-004706,Anything less than women winning 50 per cent o...,Katy Gallagher,[Q463507],2015-03-09 12:30:00,1,"[[Katy Gallagher, 0.5872], [None, 0.4128]]",[http://www.smh.com.au/act-news/women-need-to-...,E
1,2015-04-24-025718,I'd like to congratulate all the winners and f...,Helena Morrissey,[Q23762081],2015-04-24 15:33:00,1,"[[Helena Morrissey, 0.8706], [None, 0.1294]]",[http://www.cipd.co.uk/PM/peoplemanagement/b/w...,E
2,2015-07-16-044620,I think what Deepika has spoken in the video m...,Kalki Koechlin,[Q3192216],2015-07-16 16:41:07,1,"[[Kalki Koechlin, 0.6377], [None, 0.3623]]",[http://www.pinkvilla.com/entertainmenttags/ka...,E
3,2015-09-11-052815,if advocating for equal pay for equal work is ...,Hillary Clinton,[Q6294],2015-09-11 14:17:08,1,"[[Hillary Clinton, 0.8831], [None, 0.1105], [D...",[http://www.wrn.com/2015/09/hillary-clinton-ra...,E
4,2015-04-23-037713,Men and women are understandably upset if they...,Jim McDermott,"[Q321457, Q6196778]",2015-04-23 21:52:22,1,"[[Jim McDermott, 0.629], [John F. Kerry, 0.190...",[http://www.atlanticcouncil.org/en/blogs/new-a...,E


In [8]:
print(f'The dataframe has {len(df)} entries')

The dataframe has 87161 entries


In [11]:
pd.options.display.max_colwidth = 200
df.head()['quotation']

0    Anything less than women winning 50 per cent of new seats will be a loss not only for a progressive city's progress towards true gender equality but it would also be a loss for good governance in ...
1    I'd like to congratulate all the winners and finalists on their success. They have demonstrated clear leadership by moving women's progression from a `diversity' initiative to a core business prio...
2    I think what Deepika has spoken in the video makes sense. I do understand the counter argument too where everyone has been saying that had men said the same lines about having sex outside marriage...
3                                                                                    if advocating for equal pay for equal work is playing the gender card, deal me in. I am ready to play as hard as I can.
4      Men and women are understandably upset if they see a company close down and jobs lost. It's only natural people would look around and in their distress they find something o